<a href="https://colab.research.google.com/github/liveyetyeahnet/NIO/blob/main/prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%tensorflow_version 2.x
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [12]:
from google.colab import files
path_to_file = list(files.upload().keys())[0]

Saving ssq_format.csv to ssq_format (1).csv


In [13]:
df = pd.read_csv(path_to_file)
df = df.iloc[:,1:8]
print(df)

      red_1  red_2  red_3  red_4  red_5  red_6  blue_1
0        10     11     12     13     26     28      11
1         4      9     19     20     21     26      12
2         1      7     10     23     28     32      16
3         4      6      7     10     13     25       3
4         4      6     15     17     30     31      16
...     ...    ...    ...    ...    ...    ...     ...
2630      2      4      6     21     25     29       3
2631     12     15     17     24     26     31      15
2632      1      3     18     19     26     29       3
2633     16     18     19     20     29     33      12
2634      5     12     16     26     30     31      13

[2635 rows x 7 columns]


In [23]:
scaler = StandardScaler().fit(df.values)
transformed_dataset = scaler.transform(df.values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)

print(transformed_df)

             0         1         2         3         4         5         6
0     1.422875  0.296952 -0.449127 -1.178111  0.379515 -0.249260  0.520920
1    -0.197966 -0.124523  0.881019  0.144324 -0.642375 -0.775963  0.736945
2    -1.008387 -0.545999 -0.829169  0.711082  0.788270  0.804147  1.601045
3    -0.197966 -0.756736 -1.399231 -1.744869 -2.277398 -1.039314 -1.207281
4    -0.197966 -0.756736  0.120935 -0.422434  1.197026  0.540795  1.601045
...        ...       ...       ...       ...       ...       ...       ...
2630 -0.738246 -1.178212 -1.589252  0.333244  0.175137  0.014092 -1.207281
2631  1.963155  1.139903  0.500977  0.900001  0.379515  0.540795  1.385020
2632 -1.008387 -1.388949  0.690998 -0.044595  0.379515  0.014092 -1.207281
2633  3.043716  1.772116  0.881019  0.144324  0.992648  1.067498  0.736945
2634  0.072174  0.507690  0.310956  1.277840  1.197026  0.540795  0.952970

[2635 rows x 7 columns]


In [27]:
number_of_rows = df.values.shape[0]  # all our games
window_length = 30  # amount of past games we need to take in consideration for prediction
number_of_features = df.values.shape[1]  # balls count

print(number_of_rows)
print(number_of_features)

2635
7


In [28]:
train = np.empty([number_of_rows-window_length, window_length,
                  number_of_features], dtype=float)

label = np.empty([number_of_rows-window_length,
                  number_of_features], dtype=float)


for i in range(0, number_of_rows-window_length):
    train[i] = transformed_df.iloc[i:i+window_length, 0: number_of_features]
    label[i] = transformed_df.iloc[i+window_length: i +
                                   window_length+1, 0: number_of_features]

In [29]:
print(train.shape)
print(label.shape)

(2605, 30, 7)
(2605, 7)


In [30]:
print(train[0])

print(train[1])

print(label[0])

print(label[1])

[[ 1.4228748   0.2969522  -0.44912699 -1.17811072  0.37951463 -0.24925951
   0.52091979]
 [-0.19796608 -0.12452318  0.88101869  0.1443243  -0.64237486 -0.77596266
   0.73694491]
 [-1.00838652 -0.54599857 -0.82916861  0.71108217  0.78827043  0.80414677
   1.60104541]
 [-0.19796608 -0.75673627 -1.39923104 -1.74486859 -2.27739805 -1.03931423
  -1.20728121]
 [-0.19796608 -0.75673627  0.12093545 -0.42243357  1.19702623  0.5407952
   1.60104541]
 [-1.00838652 -1.38894935 -0.82916861  0.33324359  0.37951463 -0.51261109
  -0.55920584]
 [-1.00838652 -0.12452318  0.88101869  0.33324359 -0.23361906 -0.77596266
  -0.34318071]
 [ 0.07217407 -0.33526088 -1.01918942 -0.98919143 -1.45988646 -1.56601738
  -0.12715559]
 [ 0.07217407 -0.12452318  0.69099788  0.1443243  -0.43799696  0.27744363
   0.08886954]
 [-1.00838652 -1.59968704 -1.20921023 -1.17811072 -1.45988646 -1.3026658
   0.95297004]
 [-0.19796608 -0.96747396 -0.6391478  -1.36703001  1.19702623  0.80414677
   1.38502029]
 [-0.73824637  0.507689

In [31]:
batch_size = 50

model = Sequential()
model.add(LSTM(32,
               input_shape=(window_length, number_of_features),
               return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32,
               return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(number_of_features))
model.compile(loss='mse', optimizer='rmsprop')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 30, 32)            5120      
_________________________________________________________________
dropout_4 (Dropout)          (None, 30, 32)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                8320      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 7)                 231       
Total params: 13,671
Trainable params: 13,671
Non-trainable params: 0
_________________________________________________________________


In [32]:
model.fit(train, label,
          batch_size=64, epochs=1200)

Epoch 1/1200
41/41 [==============================] - 1s 24ms/step - loss: 1.0055
Epoch 2/1200
41/41 [==============================] - 1s 24ms/step - loss: 1.0019
Epoch 3/1200
41/41 [==============================] - 1s 24ms/step - loss: 1.0002
Epoch 4/1200
41/41 [==============================] - 1s 27ms/step - loss: 1.0001
Epoch 5/1200
41/41 [==============================] - 1s 26ms/step - loss: 0.9992
Epoch 6/1200
41/41 [==============================] - 1s 25ms/step - loss: 0.9982
Epoch 7/1200
41/41 [==============================] - 1s 23ms/step - loss: 0.9972
Epoch 8/1200
41/41 [==============================] - 1s 23ms/step - loss: 0.9964
Epoch 9/1200
41/41 [==============================] - 1s 24ms/step - loss: 0.9963
Epoch 10/1200
41/41 [==============================] - 1s 23ms/step - loss: 0.9961
Epoch 11/1200
41/41 [==============================] - 1s 25ms/step - loss: 0.9948
Epoch 12/1200
41/41 [==============================] - 1s 24ms/step - loss: 0.9942
Epoch 13/1200

In [35]:
to_predict = df[-30:]
print(to_predict)

      red_1  red_2  red_3  red_4  red_5  red_6  blue_1
2605      1     20     23     26     27     32      15
2606      2      8     21     25     26     30      10
2607      6      8     11     22     25     33       2
2608      5      6     11     12     15     30      12
2609      1      7     15     16     20     23       7
2610     10     17     24     25     28     30      15
2611      9     10     19     25     26     29       8
2612      4      9     10     22     28     32       8
2613      4      8     10     16     27     29       9
2614      6     14     19     20     22     24       1
2615      2      4     11     15     18     28      10
2616      3      9     11     24     25     28      16
2617     10     12     15     17     23     32       5
2618      6     13     16     20     23     32      13
2619      2      6      9     14     22     25       4
2620      6      9     17     22     24     26      16
2621      2      9     12     17     28     32       5
2622      

In [36]:
scaled_to_predict = scaler.transform(to_predict)


scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])

[ 9 14 18 20 24 26  8]
